In [1]:
import os
import sys
import pytorch_lightning as pl
import torch
import torchvision 
import torchvision.transforms as transform

sys.path.insert(0, os.path.abspath("/home/fred/Python/Personal-Project/EfficientNet/"))
import model as myModel

In [6]:
transformer = transform.Compose([
                                transform.Resize(256),
                                transform.CenterCrop(224),
                                # transform.RandomRotation(90),
                                transform.RandomHorizontalFlip(),
                                transform.RandomVerticalFlip(),
                                transform.ToTensor(),
                                transform.RandomErasing(),
                                transform.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

batch_size = 80
num_workers = 8
train_dataset = torchvision.datasets.CIFAR10("/home/fred/datasets/",train=True, transform=transformer,download=False)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size= batch_size,shuffle=True,num_workers=num_workers,pin_memory=True)

dev_dataset = torchvision.datasets.CIFAR10("/home/fred/datasets/",train=False, transform=transformer,download=False)
dev_loader = torch.utils.data.DataLoader(dev_dataset,batch_size= batch_size,shuffle=False,num_workers=num_workers,pin_memory=True)

In [7]:
class Lightning(pl.LightningModule):
    
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr = 1e-3)
    
    def training_step(self, train_batch, batch_idx):
        Xtr, Ytr = train_batch
        
        y_hat = self.model(Xtr)
        loss = torch.nn.CrossEntropyLoss()(y_hat, Ytr)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        Xval, Yval = val_batch
        
        y_hat = self.model(Xval)
        loss = torch.nn.CrossEntropyLoss()(y_hat, Yval)
        self.log('val_loss', loss)
        

In [8]:
model = Lightning(myModel.efficientNet(fi=0, num_classes=10))
trainer = pl.Trainer(gpus=1,precision=16)
trainer.fit(model,train_loader, dev_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | efficientNet | 5.7 M 
---------------------------------------
5.7 M     Trainable params
0         Non-trainable params
5.7 M     Total params
22.648    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/fred/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [9]:
torchvision.models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  